# Inicjalizacja

## Inicjalizacja zmiennych

In [ ]:
tfrecord_path_train_mal = 'drive/My Drive/Data/im_mal_dataset_train.tfrecords'
tfrecord_path_test_mal = 'drive/My Drive/Data/im_mal_dataset_test.tfrecords'

tfrecord_path_train_ben = 'drive/My Drive/Data/im_ben_dataset_train.tfrecords'
tfrecord_path_test_ben = 'drive/My Drive/Data/im_ben_dataset_test.tfrecords'

model_mal_8_path = 'drive/My Drive/Data/model_mal_8.h5'
model_mal_16_path = 'drive/My Drive/Data/model_mal_16.h5'
model_mal_32_path = 'drive/My Drive/Data/model_mal_32.h5'

model_ben_8_path = 'drive/My Drive/Data/model_ben_8.h5'
model_ben_16_path = 'drive/My Drive/Data/model_ben_16.h5'
model_ben_32_path = 'drive/My Drive/Data/model_ben_32.h5'

pickle_path_features_train_mal_8_mal = 'drive/My Drive/Data/features_train_mal_8_mal.pickle'
pickle_path_features_train_mal_16_mal = 'drive/My Drive/Data/features_train_mal_16_mal.pickle'
pickle_path_features_train_mal_32_mal = 'drive/My Drive/Data/features_train_mal_32_mal.pickle'
pickle_path_features_test_mal_8_mal = 'drive/My Drive/Data/features_test_mal_8_mal.pickle'
pickle_path_features_test_mal_16_mal = 'drive/My Drive/Data/features_test_mal_16_mal.pickle'
pickle_path_features_test_mal_32_mal = 'drive/My Drive/Data/features_test_mal_32_mal.pickle'

pickle_path_features_train_mal_8_ben = 'drive/My Drive/Data/features_train_mal_8_ben.pickle'
pickle_path_features_train_mal_16_ben = 'drive/My Drive/Data/features_train_mal_16_ben.pickle'
pickle_path_features_train_mal_32_ben = 'drive/My Drive/Data/features_train_mal_32_ben.pickle'
pickle_path_features_test_mal_8_ben = 'drive/My Drive/Data/features_test_mal_8_ben.pickle'
pickle_path_features_test_mal_16_ben = 'drive/My Drive/Data/features_test_mal_16_ben.pickle'
pickle_path_features_test_mal_32_ben = 'drive/My Drive/Data/features_test_mal_32_ben.pickle'

pickle_path_features_train_ben_8_mal = 'drive/My Drive/Data/features_train_ben_8_mal.pickle'
pickle_path_features_train_ben_16_mal = 'drive/My Drive/Data/features_train_ben_16_mal.pickle'
pickle_path_features_train_ben_32_mal = 'drive/My Drive/Data/features_train_ben_32_mal.pickle'
pickle_path_features_test_ben_8_mal = 'drive/My Drive/Data/features_test_ben_8_mal.pickle'
pickle_path_features_test_ben_16_mal = 'drive/My Drive/Data/features_test_ben_16_mal.pickle'
pickle_path_features_test_ben_32_mal = 'drive/My Drive/Data/features_test_ben_32_mal.pickle'

pickle_path_features_train_ben_8_ben = 'drive/My Drive/Data/features_train_ben_8_ben.pickle'
pickle_path_features_train_ben_16_ben = 'drive/My Drive/Data/features_train_ben_16_ben.pickle'
pickle_path_features_train_ben_32_ben = 'drive/My Drive/Data/features_train_ben_32_ben.pickle'
pickle_path_features_test_ben_8_ben = 'drive/My Drive/Data/features_test_ben_8_ben.pickle'
pickle_path_features_test_ben_16_ben = 'drive/My Drive/Data/features_test_ben_16_ben.pickle'
pickle_path_features_test_ben_32_ben = 'drive/My Drive/Data/features_test_ben_32_ben.pickle'

generate_features_mal = True
generate_features_ben = True

batch_size = 10
patch_size = 256
num_channels_input = 3
num_channels_target = 3

## Inicjalizacja bibliotek

In [ ]:
from google.colab import drive
import tensorflow as tf
import keras
import numpy as np
from keras import layers
import pickle
drive.mount('/content/drive')

Mounted at /content/drive


## Funkcje

In [ ]:
def parse_example_function(example_proto, image_feature_description):
  # Parse the input tf.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

def img_img_parse_feature_function(img_features, patch_size, num_channels_input, num_channels_target):
    input_tensor = tf.io.parse_tensor(img_features['input_image_raw'], out_type=tf.float32)
    target_tensor = tf.io.parse_tensor(img_features['target_image_raw'], out_type=tf.float32)   
    input_tensor.set_shape([patch_size, patch_size, num_channels_input])
    target_tensor.set_shape([patch_size, patch_size, num_channels_target])
    return input_tensor, target_tensor

def tfrecord_2_img_img_dataset(tfrecord_path, batch_size, patch_size, num_channels_input=3, num_channels_target=1, num_prefatch=1, num_repeats=1):
    
    image_feature_description = {
        'input_image_raw': tf.io.FixedLenFeature([], tf.string),
        'target_image_raw': tf.io.FixedLenFeature([], tf.string)
    }
    
    _parse_example = lambda example: parse_example_function(example, image_feature_description)
    _parse_feature = lambda feature: img_img_parse_feature_function(feature, patch_size, num_channels_input, num_channels_target)

    dataset = tf.data.TFRecordDataset(tfrecord_path)
    dataset = dataset.map(_parse_example)
    dataset = dataset.map(_parse_feature)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(num_prefatch)
    dataset = dataset.repeat(num_repeats)
    return dataset

# Ładowanie danych

## Ładowanie modeli

In [ ]:
autoencoder_model_mal_8 = keras.models.load_model(model_mal_8_path)
autoencoder_model_mal_16 = keras.models.load_model(model_mal_16_path)
autoencoder_model_mal_32 = keras.models.load_model(model_mal_32_path)
autoencoder_model_ben_8 = keras.models.load_model(model_ben_8_path)
autoencoder_model_ben_16 = keras.models.load_model(model_ben_16_path)
autoencoder_model_ben_32 = keras.models.load_model(model_ben_32_path)

## Ładowanie TFRecords

In [ ]:
dset_train_ben = tfrecord_2_img_img_dataset(tfrecord_path_train_ben, batch_size, patch_size, num_channels_input, num_channels_target, num_repeats = 1)
dset_test_ben = tfrecord_2_img_img_dataset(tfrecord_path_test_ben, batch_size, patch_size, num_channels_input, num_channels_target, num_repeats = 1)

dset_train_mal = tfrecord_2_img_img_dataset(tfrecord_path_train_mal, batch_size, patch_size, num_channels_input, num_channels_target, num_repeats = 1)
dset_test_mal = tfrecord_2_img_img_dataset(tfrecord_path_test_mal, batch_size, patch_size, num_channels_input, num_channels_target, num_repeats = 1)

# Generacja cech autoenkodera

In [ ]:
feature_extractor_mal_8 = keras.Model(
    inputs=autoencoder_model_mal_8.inputs,
    outputs=autoencoder_model_mal_8.get_layer(index=9).output,
)

feature_extractor_mal_16 = keras.Model(
    inputs=autoencoder_model_mal_16.inputs,
    outputs=autoencoder_model_mal_16.get_layer(index=9).output,
)

feature_extractor_mal_32 = keras.Model(
    inputs=autoencoder_model_mal_32.inputs,
    outputs=autoencoder_model_mal_32.get_layer(index=9).output,
)

feature_extractor_ben_8 = keras.Model(
    inputs=autoencoder_model_ben_8.inputs,
    outputs=autoencoder_model_ben_8.get_layer(index=9).output,
)

feature_extractor_ben_16 = keras.Model(
    inputs=autoencoder_model_ben_16.inputs,
    outputs=autoencoder_model_ben_16.get_layer(index=9).output,
)

feature_extractor_ben_32 = keras.Model(
    inputs=autoencoder_model_ben_32.inputs,
    outputs=autoencoder_model_ben_32.get_layer(index=9).output,
)

In [ ]:
if generate_features_mal:
  feature_train_mal_8_mal = feature_extractor_mal_8.predict(dset_train_mal)
  feature_train_mal_8_mal = feature_train_mal_8_mal.reshape(feature_train_mal_8_mal.shape[0], -1)
  with open(pickle_path_features_train_mal_8_mal, 'wb') as handle:
      pickle.dump(feature_train_mal_8_mal, handle)

  feature_train_mal_8_ben = feature_extractor_mal_8.predict(dset_train_ben)
  feature_train_mal_8_ben = feature_train_mal_8_ben.reshape(feature_train_mal_8_ben.shape[0], -1)
  with open(pickle_path_features_train_mal_8_ben, 'wb') as handle:
      pickle.dump(feature_train_mal_8_ben, handle)

  feature_train_mal_16_mal = feature_extractor_mal_16.predict(dset_train_mal)
  feature_train_mal_16_mal = feature_train_mal_16_mal.reshape(feature_train_mal_16_mal.shape[0], -1)
  with open(pickle_path_features_train_mal_16_mal, 'wb') as handle:
      pickle.dump(feature_train_mal_16_mal, handle)

  feature_train_mal_16_ben = feature_extractor_mal_16.predict(dset_train_ben)
  feature_train_mal_16_ben = feature_train_mal_16_ben.reshape(feature_train_mal_16_ben.shape[0], -1)
  with open(pickle_path_features_train_mal_16_ben, 'wb') as handle:
      pickle.dump(feature_train_mal_16_ben, handle)

  feature_train_mal_32_mal = feature_extractor_mal_32.predict(dset_train_mal)
  feature_train_mal_32_mal = feature_train_mal_32_mal.reshape(feature_train_mal_32_mal.shape[0], -1)
  with open(pickle_path_features_train_mal_32_mal, 'wb') as handle:
      pickle.dump(feature_train_mal_32_mal, handle)

  feature_train_mal_32_ben = feature_extractor_mal_32.predict(dset_train_ben)
  feature_train_mal_32_ben = feature_train_mal_32_ben.reshape(feature_train_mal_32_ben.shape[0], -1)
  with open(pickle_path_features_train_mal_32_ben, 'wb') as handle:
      pickle.dump(feature_train_mal_32_ben, handle)

In [ ]:
if generate_features_mal:
  feature_test_mal_8_mal = feature_extractor_mal_8.predict(dset_test_mal)
  feature_test_mal_8_mal = feature_test_mal_8_mal.reshape(feature_test_mal_8_mal.shape[0], -1)
  with open(pickle_path_features_test_mal_8_mal, 'wb') as handle:
      pickle.dump(feature_test_mal_8_mal, handle)

  feature_test_mal_8_ben = feature_extractor_mal_8.predict(dset_test_ben)
  feature_test_mal_8_ben = feature_test_mal_8_ben.reshape(feature_test_mal_8_ben.shape[0], -1)
  with open(pickle_path_features_test_mal_8_ben, 'wb') as handle:
      pickle.dump(feature_test_mal_8_ben, handle)

  feature_test_mal_16_mal = feature_extractor_mal_16.predict(dset_test_mal)
  feature_test_mal_16_mal = feature_test_mal_16_mal.reshape(feature_test_mal_16_mal.shape[0], -1)
  with open(pickle_path_features_test_mal_16_mal, 'wb') as handle:
      pickle.dump(feature_test_mal_16_mal, handle)

  feature_test_mal_16_ben = feature_extractor_mal_16.predict(dset_test_ben)
  feature_test_mal_16_ben = feature_test_mal_16_ben.reshape(feature_test_mal_16_ben.shape[0], -1)
  with open(pickle_path_features_test_mal_16_ben, 'wb') as handle:
      pickle.dump(feature_test_mal_16_ben, handle)

  feature_test_mal_32_mal = feature_extractor_mal_32.predict(dset_test_mal)
  feature_test_mal_32_mal = feature_test_mal_32_mal.reshape(feature_test_mal_32_mal.shape[0], -1)
  with open(pickle_path_features_test_mal_32_mal, 'wb') as handle:
      pickle.dump(feature_test_mal_32_mal, handle)

  feature_test_mal_32_ben = feature_extractor_mal_32.predict(dset_test_ben)
  feature_test_mal_32_ben = feature_test_mal_32_ben.reshape(feature_test_mal_32_ben.shape[0], -1)
  with open(pickle_path_features_test_mal_32_ben, 'wb') as handle:
      pickle.dump(feature_test_mal_32_ben, handle)

In [ ]:
if generate_features_ben:
  feature_train_ben_8_mal = feature_extractor_ben_8.predict(dset_train_mal)
  feature_train_ben_8_mal = feature_train_ben_8_mal.reshape(feature_train_ben_8_mal.shape[0], -1)
  with open(pickle_path_features_train_ben_8_mal, 'wb') as handle:
      pickle.dump(feature_train_ben_8_mal, handle)

  feature_train_ben_8_ben = feature_extractor_ben_8.predict(dset_train_ben)
  feature_train_ben_8_ben = feature_train_ben_8_ben.reshape(feature_train_ben_8_ben.shape[0], -1)
  with open(pickle_path_features_train_ben_8_ben, 'wb') as handle:
      pickle.dump(feature_train_ben_8_ben, handle)

  feature_train_ben_16_mal = feature_extractor_ben_16.predict(dset_train_mal)
  feature_train_ben_16_mal = feature_train_ben_16_mal.reshape(feature_train_ben_16_mal.shape[0], -1)
  with open(pickle_path_features_train_ben_16_mal, 'wb') as handle:
      pickle.dump(feature_train_ben_16_mal, handle)

  feature_train_ben_16_ben = feature_extractor_ben_16.predict(dset_train_ben)
  feature_train_ben_16_ben = feature_train_ben_16_ben.reshape(feature_train_ben_16_ben.shape[0], -1)
  with open(pickle_path_features_train_ben_16_ben, 'wb') as handle:
      pickle.dump(feature_train_ben_16_ben, handle)

  feature_train_ben_32_mal = feature_extractor_ben_32.predict(dset_train_mal)
  feature_train_ben_32_mal = feature_train_ben_32_mal.reshape(feature_train_ben_32_mal.shape[0], -1)
  with open(pickle_path_features_train_ben_32_mal, 'wb') as handle:
      pickle.dump(feature_train_ben_32_mal, handle)

  feature_train_ben_32_ben = feature_extractor_ben_32.predict(dset_train_ben)
  feature_train_ben_32_ben = feature_train_ben_32_ben.reshape(feature_train_ben_32_ben.shape[0], -1)
  with open(pickle_path_features_train_ben_32_ben, 'wb') as handle:
      pickle.dump(feature_train_ben_32_ben, handle)

In [ ]:
if generate_features_ben:
  feature_test_ben_8_mal = feature_extractor_ben_8.predict(dset_test_mal)
  feature_test_ben_8_mal = feature_test_ben_8_mal.reshape(feature_test_ben_8_mal.shape[0], -1)
  with open(pickle_path_features_test_ben_8_mal, 'wb') as handle:
      pickle.dump(feature_test_ben_8_mal, handle)

  feature_test_ben_8_ben = feature_extractor_ben_8.predict(dset_test_ben)
  feature_test_ben_8_ben = feature_test_ben_8_ben.reshape(feature_test_ben_8_ben.shape[0], -1)
  with open(pickle_path_features_test_ben_8_ben, 'wb') as handle:
      pickle.dump(feature_test_ben_8_ben, handle)

  feature_test_ben_16_mal = feature_extractor_ben_16.predict(dset_test_mal)
  feature_test_ben_16_mal = feature_test_ben_16_mal.reshape(feature_test_ben_16_mal.shape[0], -1)
  with open(pickle_path_features_test_ben_16_mal, 'wb') as handle:
      pickle.dump(feature_test_ben_16_mal, handle)

  feature_test_ben_16_ben = feature_extractor_ben_16.predict(dset_test_ben)
  feature_test_ben_16_ben = feature_test_ben_16_ben.reshape(feature_test_ben_16_ben.shape[0], -1)
  with open(pickle_path_features_test_ben_16_ben, 'wb') as handle:
      pickle.dump(feature_test_ben_16_ben, handle)

  feature_test_ben_32_mal = feature_extractor_ben_32.predict(dset_test_mal)
  feature_test_ben_32_mal = feature_test_ben_32_mal.reshape(feature_test_ben_32_mal.shape[0], -1)
  with open(pickle_path_features_test_ben_32_mal, 'wb') as handle:
      pickle.dump(feature_test_ben_32_mal, handle)

  feature_test_ben_32_ben = feature_extractor_ben_32.predict(dset_test_ben)
  feature_test_ben_32_ben = feature_test_ben_32_ben.reshape(feature_test_ben_32_ben.shape[0], -1)
  with open(pickle_path_features_test_ben_32_ben, 'wb') as handle:
      pickle.dump(feature_test_ben_32_ben, handle)